# Random Forest from scratch for Iris dataset

In [1]:
import numpy as np

from sklearn.datasets import load_iris # to load dataset
from sklearn.model_selection import train_test_split # to split dataset to train and test parts
from sklearn.metrics import f1_score # to evaluate f1_score metric

## Functions

"Thoughts" of one tree:

In [2]:
def Entropy(P_left,P_right,w_left,w_right):
    dH = 0

    for p in P_left:
        if (p!=0): 
            dH += - w_left*np.log2(p)

    for p in P_right:
        if (p!=0): 
            dH +=  - w_right*np.log2(p)

    return dH

def choose_nodes(H_before,X,Y,inf_gain_threshold,len_features):
    inf_gain_max = 0
    possible_node = None
    for feature in range(len_features):
        feature_values = np.array([X[i][feature] for i in range(len(X))])
        if max(feature_values) != min(feature_values):
            step = (max(feature_values) - min(feature_values)) / np.random.randint(2,10)
            thresholds = np.arange(min(feature_values)+step,max(feature_values),step)
            for threshold in thresholds:
                X_left = []
                X_right = []
                Y_left = []
                Y_right = []
                for i in range(len(X)):
                    if X[i][feature] < threshold:
                        X_left.append(X[i])
                        Y_left.append(Y[i])
                    else:
                        X_right.append(X[i])
                        Y_right.append(Y[i])
                if len(X) > 0:
                    w_left = len(X_left) / len(X)
                    w_right = 1 - w_left
                    P_left = P(Y_left)
                    P_right = P(Y_right)
                    H = Entropy(P_left,P_right,w_left,w_right)
                    inf_gain = H_before - H
                    if inf_gain > inf_gain_max:
                        inf_gain_max = inf_gain
                        possible_node = [inf_gain,H,X_left,Y_left,X_right,Y_right,feature,threshold]
    if possible_node:
        inf_gain,H,X_left,Y_left,X_right,Y_right,feature,threshold = possible_node
        if inf_gain > inf_gain_threshold:
            return possible_node
    else:
        return None

def P(Y):
    Y = np.array(Y)
    Y_set = list(np.unique(Y))
    if len(Y_set) > 1:
        P = np.zeros(len(Y_set))
        for i in range(len(Y_set)):
            y = Y_set[i]
            P[i] = len(list(Y[Y==y])) / len(list(Y))
    elif len(Y_set) == 1:
        P = np.array([1])
    else:
        P = np.array([0])
    return P

def max_category(Y):
    Y = np.array(Y,dtype=np.int16)
    return np.argmax(np.bincount(Y))

def create_node(H,X,Y,feature,threshold,left,right,up,end=False,prediction=None):
    node = {}
    node['Entropy'] = H
    node['X'] = X
    node['Y'] = Y
    node['feature'] = feature
    node['threshold'] = threshold
    node['left'] = left
    node['right'] = right
    node['up'] = up
    node['END'] = end
    node['prediction'] = prediction
    return node

Tree training:

In [3]:
def train_tree(X_train,Y_train,inf_gain_threshold,len_features):

    if not len_features:
        len_features=X_train.shape[1]
        
    tree = {}
    w_left = 1
    w_right = 0
    P_left = P(Y_train)
    P_right = np.array([0])
    H = Entropy(P_left,P_right,w_left,w_right)

    tree['top'] = create_node(H,X_train,Y_train,feature=None,threshold=None,left=None,right=None,up=None,end=False,prediction=None)
    queue_list = ['top']

    while queue_list:

        current = queue_list.pop()
        X_current = tree[current]['X']
        Y_current = tree[current]['Y']
        possible_node = choose_nodes(H,X_current,Y_current,len_features=len_features,inf_gain_threshold=inf_gain_threshold)
        
        if possible_node:
            inf_gain,H,X_left,Y_left,X_right,Y_right,feature,threshold = possible_node
            if (len(X_left) > 0 and len(X_right) > 0) :
                right = current + 'right'
                tree[right] = create_node(H,X_right,Y_right,feature=None,threshold=None,left=None,right=None,up=current,end=False,prediction=None)
                queue_list.append(right)

                left = current + 'left'
                tree[left] = create_node(H,X_left,Y_left,feature=None,threshold=None,left=None,right=None,up=current,end=False,prediction=None)
                queue_list.append(left)

                tree[current]['feature'] = feature
                tree[current]['threshold'] = threshold
                tree[current]['left'] = left
                tree[current]['right'] = right

            else:
                tree[current]['END'] = True
                tree[current]['prediction'] = max_category(Y_current)

        else:
            tree[current]['END'] = True
            tree[current]['prediction'] = max_category(Y_current)

    training_tree = {}

    for key,value in list(tree.items()):
        training_tree[key] = create_node(value['Entropy'],None,None,feature=value['feature'],threshold=value['threshold'],left=value['left'],right=value['right'],up=None,end=value['END'],prediction=value['prediction'])
    
    return training_tree

Tree prediction:

In [4]:
def predict(x,tree):
    current = 'top'
    prediction = None
    while True:
        end = tree[current]['END']
        if end:
            prediction = tree[current]['prediction']
            break
        else:
            feature = tree[current]['feature']
            threshold = tree[current]['threshold']
            value = x[feature]
            if value < threshold:
                current = current + 'left'
            else:
                current = current + 'right'
    return prediction

Accuracy evaluation:

In [5]:
def val_acc(X_test,Y_test,tree):
    count = 0
    for x,y in zip(X_test,Y_test):
        y_pred = predict(x,tree)
        if y_pred == y:
            count += 1
    acc = count/len(Y_test)
    return acc

F1_score evaluation:

In [6]:
def val_f1(X_test,Y_test,tree):
    Y_pred = np.zeros_like(Y_test)
    i=0
    for x in X_test:
        y_pred = predict(x,tree)
        Y_pred[i] = y_pred
        i+=1
    f1 = f1_score(Y_test,Y_pred,average='weighted')
    return f1

Random Forest training:

In [7]:
def train_random_forest(X_train_set,Y_train_set,inf_gain_threshold,len_features):
    forest = []
    for i in range(len(X_train_set)):
        x_train = X_train_set[i]
        y_train = Y_train_set[i]
        tree = train_tree(x_train,y_train,inf_gain_threshold=inf_gain_threshold,len_features=len_features)
        forest.append(tree)
    return forest

Random Forest prediction:

In [8]:
def predict_random_forest(X,forest):
    Y_pred = []
    for x in X:
        Y = []
        for tree in forest:
            y = predict(x,tree)
            Y.append(y)
        y_pred = np.argmax(np.bincount(Y))
        Y_pred.append(y_pred)
    return Y_pred

Random Forest accuracy evaluation:

In [9]:
def val_acc_random_forest(X_test,Y_test,forest):
    count = 0
    for x,y in zip(X_test,Y_test):
        Y_pred = []
        for tree in forest:
            y_pred = predict(x,tree)
            Y_pred.append(y_pred)
        y_pred = np.argmax(np.bincount(Y_pred))
        if y_pred == y:
            count += 1
    acc = count/len(Y_test)
    return acc

Random Forest F1_score evaluation:

In [10]:
def val_f1_random_forest(X_test,Y_test,forest):
    Y_pred = np.zeros_like(Y_test)
    i=0
    for x in X_test:
        Y = []
        for tree in forest:
            y_pred = predict(x,tree)
            Y.append(y_pred)
        y_pred = np.argmax(np.bincount(Y))
        Y_pred[i] = y_pred
        i+=1
    f1 = f1_score(Y_test,Y_pred,average='weighted')
    return f1

Datasets creation:

In [11]:
def create_train_sets(X_train,Y_train,len_set,num_trees):
    X_train_set = [0]*num_trees
    Y_train_set = [0]*num_trees
    for i in range(num_trees):
        x_train_set = np.zeros([len_set,X_train.shape[1]])
        y_train_set = np.zeros([len_set])
        for j in range(len(x_train_set)):
            number = np.random.randint(0,len(X_train))
            x_train_set[j] = X_train[number]
            y_train_set[j] = Y_train[number]
        X_train_set[i] = x_train_set
        Y_train_set[i] = y_train_set
    return X_train_set, Y_train_set

Random forest training with evaluation:

In [12]:
def train_best_random_forest(X_train,Y_train,X_val,Y_val,inf_gain_threshold,len_features,num_trees,iterations):
    f1_best = 0
    for _ in range(iterations):
        X_train_set, Y_train_set = create_train_sets(X_train,Y_train,len_set=int(np.sqrt(len(X_train))),num_trees=num_trees)
        forest_i = train_random_forest(X_train_set,Y_train_set,inf_gain_threshold,len_features)
        f1_i = float(val_f1_random_forest(X_val,Y_val,forest_i))
        if f1_i > f1_best:
            forest = forest_i
    return forest

## Loading data and training

In [13]:
data = load_iris()
X, Y = data['data'], data['target']
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3)
X_train, X_test_val, Y_train, Y_test_val = train_test_split(X,Y,test_size=0.4)
X_val, X_test, Y_val, Y_test = train_test_split(X_test_val,Y_test_val,test_size=0.5)

### One Tree

In [14]:
tree = train_tree(X_train,Y_train,inf_gain_threshold=0.1,len_features=X_train.shape[1])

Tree structure:

In [15]:
tree

{'top': {'Entropy': 4.756491387866058,
  'X': None,
  'Y': None,
  'feature': 2,
  'threshold': 1.9500000000000002,
  'left': 'topleft',
  'right': 'topright',
  'up': None,
  'END': False,
  'prediction': None},
 'topright': {'Entropy': 1.3558183768550345,
  'X': None,
  'Y': None,
  'feature': None,
  'threshold': None,
  'left': None,
  'right': None,
  'up': None,
  'END': True,
  'prediction': 2},
 'topleft': {'Entropy': 1.3558183768550345,
  'X': None,
  'Y': None,
  'feature': 0,
  'threshold': 4.680000000000001,
  'left': 'topleftleft',
  'right': 'topleftright',
  'up': None,
  'END': False,
  'prediction': None},
 'topleftright': {'Entropy': 0.0,
  'X': None,
  'Y': None,
  'feature': None,
  'threshold': None,
  'left': None,
  'right': None,
  'up': None,
  'END': True,
  'prediction': 0},
 'topleftleft': {'Entropy': 0.0,
  'X': None,
  'Y': None,
  'feature': None,
  'threshold': None,
  'left': None,
  'right': None,
  'up': None,
  'END': True,
  'prediction': 0}}

In [16]:
print('Accuracy = ',round(val_acc(X_test,Y_test,tree),2))
print('F1 score = ',round(val_f1(X_test,Y_test,tree),2))

Accuracy =  0.57
F1 score =  0.45


## Random Forest:

In [17]:
X_train_set, Y_train_set = create_train_sets(X_train,Y_train,len_set=len(X_train),num_trees=128)

In [18]:
forest = train_random_forest(X_train_set,Y_train_set,0.01,4)

In [19]:
print('Accuracy = ',round(val_acc_random_forest(X_test,Y_test,forest),2))
print('F1 score = ',round(val_f1_random_forest(X_test,Y_test,forest),2))

Accuracy =  0.57
F1 score =  0.45


## Improved Random Forest with evaluation during training

In [20]:
forest = train_best_random_forest(X_train,Y_train,X_val,Y_val,0.01,X_train.shape[1],num_trees=100,iterations=10)

In [21]:
print('Accuracy = ',round(val_acc_random_forest(X_test,Y_test,forest),2))
print('F1 score = ',round(val_f1_random_forest(X_test,Y_test,forest),2))

Accuracy =  0.97
F1 score =  0.97
